# Inference segmentation with classical CV

Load model from [segmentation models](https://github.com/qubvel-org/segmentation_models.pytorch) and process some test images

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import os

In [ ]:
import numpy as np
import time
import typing as tp

import matplotlib.pyplot as plt
import cv2
import onnxruntime as ort

In [ ]:
IMAGE_DIR = '../test_images'
DEVICE = 'cpu'
NN_THRESHOLD = 0.7
BATCH_SIZE = 1

In [ ]:
def onnx_preprocessing(
    image: np.ndarray,
    image_size: tp.Tuple[int, int] = (224, 224),
) -> np.ndarray:
    # resize
    image = cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_LINEAR)

    # normalize
    mean = np.array((0.485, 0.456, 0.406), dtype=np.float32) * 255.0
    std = np.array((0.229, 0.224, 0.225), dtype=np.float32) * 255.0
    denominator = np.reciprocal(std, dtype=np.float32)
    image = image.astype(np.float32)
    image -= mean
    image *= denominator

    # transpose
    image = image.transpose((2, 0, 1))[None]
    return image


def binary_mask_iou(
        mask1: np.array,
        mask2: np.array,
        label_value: int = 1,
) -> float:
    mask1_area = np.count_nonzero(mask1 == label_value)
    mask2_area = np.count_nonzero(mask2 == label_value)
    intersection = np.count_nonzero(
        np.logical_and(mask1==label_value,  mask2==label_value)
    )
    return intersection / (mask1_area + mask2_area-intersection)

In [ ]:
import json
from tqdm import tqdm

# Process annotations data
d_path_annot = '../IoU_test/annotations/balanced_test.json'
d_path_images = '../IoU_test/images'

processed_data = {
    'Id': [],
    'image_path': [],
    'semantic_masks': [],
    'granules_number': []
}

with open(d_path_annot, 'r', encoding="utf-8") as json_file:
    json_data_dir = json.load(json_file)
    # Process image data
    for image_inf in tqdm(json_data_dir['images'], desc="Process images: "):
        real_img_id = image_inf['id']
        for k in processed_data:
            processed_data[k].append([])
        processed_data['Id'][-1] = real_img_id
        img_name = image_inf['file_name'].split('/')[-1]
        img_path = os.path.join(
            d_path_images, img_name
        )
        processed_data['image_path'][-1] = str(img_path)
        SIZE = (image_inf['height'], image_inf['width'], 3)
        processed_data['semantic_masks'][-1] = np.zeros(SIZE[:2], dtype=np.uint8)
        processed_data['granules_number'][-1] = 0

with open(d_path_annot, 'r') as json_file:
    json_data_dir = json.load(json_file)
    image_id_old = ''
    skipped_counter = 1
    morph_kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3, 3)
    )

    for annotation_data in tqdm(
        json_data_dir['annotations'], desc="Process annotations: "
    ):
        # Data may be numerated to image of have through numeration
        process_image_id = annotation_data['image_id']

        image_data_indx = processed_data['Id'].index(process_image_id)
        label = annotation_data['category_id']
        # Process each granule
        for point_i, point in enumerate(annotation_data['segmentation']):
            if isinstance(point, list):
                point_xy = [
                    [point[j], point[j + 1]] for j in
                    range(0, len(point), 2)
                ]
                cnt = np.array(point_xy).reshape((-1, 1, 2)).astype(
                    np.int32
                )
                if len(cnt) < 3:  # bad contour
                    print('Cnt is bad')
                    continue

                single_mask = np.zeros((480, 480), dtype=np.uint8)
                _ = cv2.drawContours(
                    single_mask,
                    [cnt], -1, label, cv2.FILLED
                )
                processed_data['semantic_masks'][image_data_indx][single_mask != 0] = label
                x, y, w, h = annotation_data["bbox"]
                processed_data['granules_number'][image_data_indx] += 1
            else:
                continue

In [ ]:
# Test valid
import random
random_i = random.randint(0, len(processed_data['Id'])-1)

random_mask_all = processed_data['semantic_masks'][random_i]
img = random_mask_all.copy()
img[img > 0] = 255

print((
    f"Image: {processed_data['image_path'][random_i]},\n"
    f" total granules: {processed_data['granules_number'][random_i]}.\n"
))


test_image = cv2.imread(processed_data['image_path'][random_i])
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

plt.rcParams['figure.figsize'] = [10, 20]
f, axarr = plt.subplots(2,1)

axarr[0].imshow(test_image, cmap='gray', vmin=0, vmax=255)
axarr[1].imshow(img, cmap='gray', vmin=0, vmax=255)

## Unet
 
[Unet](https://arxiv.org/abs/1505.04597) + [timm-mobilenetv3_small_minimal_100](https://smp.readthedocs.io/en/latest/encoders.html#mobilenet) + [`cv2.findContours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0)

In [ ]:
providers = [
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    '../models/UNet-MobileNetV3-large-075-200.onnx',
    providers=providers
)

In [ ]:
# Simple contours find
for image_path in list(os.walk(IMAGE_DIR + '/preprocess/'))[0][2]:
    if 'png' not in image_path:
        continue
    print('process: ', image_path)
    test_image = cv2.imread(IMAGE_DIR + '/preprocess/' + image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 255
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    for cnt in cnts:
        x1,y1,w,h = cv2.boundingRect(cnt)
        test_image = cv2.rectangle(
            test_image,
            (int(x1), int(y1)), (int(x1+w), int(y1+h)),
            (255,0,0),
            2
        )

    green_masks = np.zeros(test_image.shape, np.uint8)
    green_masks[:,:,0][pr_mask > 0] = 61
    green_masks[:,:,1][pr_mask > 0] = 142
    green_masks[:,:,2][pr_mask > 0] = 48

    test_image = cv2.addWeighted(test_image, 0.6, green_masks, 0.4, 0)
    _ = cv2.imwrite(IMAGE_DIR + f'/final/Unet-mobilenet_{int(NN_THRESHOLD*10)}_' + image_path, cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    time.sleep(1)

    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.imshow(test_image)

In [ ]:
total_iou = []
total_granules_det = []

for image_i, image_path in enumerate(tqdm(processed_data['image_path'])):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    gt_mask = processed_data['semantic_masks'][image_i]

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 1
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    total_granules_det.append(len(cnts))
    total_iou.append(binary_mask_iou(gt_mask, pr_mask))

granules_deviation = np.abs((np.array(total_granules_det) - np.array(processed_data['granules_number']))) / np.array(processed_data['granules_number']) * 100
granules_detected = (np.array(total_granules_det) / np.array(processed_data['granules_number'])) * 100


print(f"Median and Average IoU for all dataset = {np.median(total_iou)} VS {np.average(total_iou)}, disp = {np.std(total_iou)}")
print(f"Median and Average granules devianion for all dataset = {np.median(granules_deviation)} VS {np.average(granules_deviation)}, disp = {np.std(granules_deviation)}")
print(f"Median and Average granules detected for all dataset = {np.median(granules_detected)} VS {np.average(granules_deviation)}, disp = {np.std(granules_detected)}")

## Unet

[Unet](https://arxiv.org/abs/1505.04597) + ResNet50 + [`cv2.findContours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0)

In [ ]:
providers = [
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    '../models/UNet-ResNet50.onnx',
    providers=providers
)

In [ ]:
# Simple contours find
for image_path in list(os.walk(IMAGE_DIR + '/preprocess/'))[0][2]:
    if 'png' not in image_path:
        continue
    print('process: ', image_path)
    test_image = cv2.imread(IMAGE_DIR + '/preprocess/' + image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 255
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    for cnt in cnts:
        x1,y1,w,h = cv2.boundingRect(cnt)
        test_image = cv2.rectangle(
            test_image,
            (int(x1), int(y1)), (int(x1+w), int(y1+h)),
            (255,0,0),
            2
        )

    green_masks = np.zeros(test_image.shape, np.uint8)
    green_masks[:,:,0][pr_mask > 0] = 61
    green_masks[:,:,1][pr_mask > 0] = 142
    green_masks[:,:,2][pr_mask > 0] = 48

    test_image = cv2.addWeighted(test_image, 0.6, green_masks, 0.4, 0)
    _ = cv2.imwrite(IMAGE_DIR + f'/final/Unet-ResNet50_{int(NN_THRESHOLD*10)}_' + image_path, cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    time.sleep(1)

    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.imshow(test_image)

In [ ]:
total_iou = []
total_granules_det = []

for image_i, image_path in enumerate(tqdm(processed_data['image_path'])):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    gt_mask = processed_data['semantic_masks'][image_i]

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 1
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    total_granules_det.append(len(cnts))
    total_iou.append(binary_mask_iou(gt_mask, pr_mask))

granules_deviation = np.abs((np.array(total_granules_det) - np.array(processed_data['granules_number']))) / np.array(processed_data['granules_number']) * 100
granules_detected = (np.array(total_granules_det) / np.array(processed_data['granules_number'])) * 100


print(f"Median and Average IoU for all dataset = {np.median(total_iou)} VS {np.average(total_iou)}, disp = {np.std(total_iou)}")
print(f"Median and Average granules devianion for all dataset = {np.median(granules_deviation)} VS {np.average(granules_deviation)}, disp = {np.std(granules_deviation)}")
print(f"Median and Average granules detected for all dataset = {np.median(granules_detected)} VS {np.average(granules_detected)}, disp = {np.std(granules_detected)}")

## FPN

[FPN](http://presentations.cocodataset.org/COCO17-Stuff-FAIR.pdf) + ResNet50 + [`cv2.findContours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0)

In [ ]:
providers = [
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    '../models/FPN-ResNet50.onnx',
    providers=providers
)

In [ ]:
# Simple contours find
for image_path in list(os.walk(IMAGE_DIR + '/preprocess/'))[0][2]:
    test_image = cv2.imread(IMAGE_DIR + '/preprocess/' + image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    # pr_mask = (pr_mask.squeeze().cpu().detach().numpy().round())
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 255
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    for cnt in cnts:
        x1,y1,w,h = cv2.boundingRect(cnt)
        test_image = cv2.rectangle(
            test_image,
            (int(x1), int(y1)), (int(x1+w), int(y1+h)),
            (255,0,0),
            2
        )

    green_masks = np.zeros(test_image.shape, np.uint8)
    green_masks[:,:,0][pr_mask > 0] = 61
    green_masks[:,:,1][pr_mask > 0] = 142
    green_masks[:,:,2][pr_mask > 0] = 48

    test_image = cv2.addWeighted(test_image, 0.6, green_masks, 0.4, 0)
    _ = cv2.imwrite(IMAGE_DIR + f'/final/FPN_{int(NN_THRESHOLD*10)}_' + image_path, cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    time.sleep(1)

    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.imshow(test_image)

In [ ]:
total_iou = []
total_granules_det = []

for image_i, image_path in enumerate(tqdm(processed_data['image_path'])):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    gt_mask = processed_data['semantic_masks'][image_i]

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 1
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    total_granules_det.append(len(cnts))
    total_iou.append(binary_mask_iou(gt_mask, pr_mask))

granules_deviation = np.abs((np.array(total_granules_det) - np.array(processed_data['granules_number']))) / np.array(processed_data['granules_number']) * 100
granules_detected = (np.array(total_granules_det) / np.array(processed_data['granules_number'])) * 100


print(f"Median and Average IoU for all dataset = {np.median(total_iou)} VS {np.average(total_iou)}, disp = {np.std(total_iou)}")
print(f"Median and Average granules devianion for all dataset = {np.median(granules_deviation)} VS {np.average(granules_deviation)}, disp = {np.std(granules_deviation)}")
print(f"Median and Average granules detected for all dataset = {np.median(granules_detected)} VS {np.average(granules_detected)}, disp = {np.std(granules_detected)}")

## MAnet

[MAnet](https://ieeexplore.ieee.org/abstract/document/9201310) + [timm-mobilenetv3_large_100](https://smp.readthedocs.io/en/latest/encoders.html#mobilenet) + [`cv2.findContours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0)

In [ ]:
providers = [
    'CUDAExecutionProvider',
    'CPUExecutionProvider',
]

ort_session = ort.InferenceSession(
    '../models/MAnet-ResNet50.onnx',
    providers=providers
)

In [ ]:
# Simple contours find
for image_path in list(os.walk(IMAGE_DIR + '/preprocess/'))[0][2]:
    test_image = cv2.imread(IMAGE_DIR + '/preprocess/' + image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 255
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    for cnt in cnts:
        x1,y1,w,h = cv2.boundingRect(cnt)
        test_image = cv2.rectangle(
            test_image,
            (int(x1), int(y1)), (int(x1+w), int(y1+h)),
            (255,0,0),
            2
        )

    green_masks = np.zeros(test_image.shape, np.uint8)
    green_masks[:,:,0][pr_mask > 0] = 61
    green_masks[:,:,1][pr_mask > 0] = 142
    green_masks[:,:,2][pr_mask > 0] = 48

    test_image = cv2.addWeighted(test_image, 0.6, green_masks, 0.4, 0)
    _ = cv2.imwrite(IMAGE_DIR + f'/final/MAnet_{int(NN_THRESHOLD*10)}_' + image_path, cv2.cvtColor(test_image, cv2.COLOR_RGB2BGR))
    time.sleep(1)

    fig, ax = plt.subplots(figsize=(10, 10))
    _ = ax.imshow(test_image)

In [ ]:
total_iou = []
total_granules_det = []

for image_i, image_path in enumerate(tqdm(processed_data['image_path'])):
    test_image = cv2.imread(image_path)
    test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
    gt_mask = processed_data['semantic_masks'][image_i]

    onnx_input = onnx_preprocessing(test_image, image_size=(480, 480))
    onnx_input = np.concatenate([onnx_input] * BATCH_SIZE)
    ort_inputs = {ort_session.get_inputs()[0].name: onnx_input}
    ort_outputs = ort_session.run(None, ort_inputs)[0]

    pr_mask = ort_outputs.squeeze().round()
    pr_mask = np.exp(-np.logaddexp(0, -pr_mask))  # sigmoid
    pr_mask[pr_mask >= NN_THRESHOLD] = 1
    pr_mask[pr_mask < NN_THRESHOLD] = 0
    pr_mask = pr_mask.astype(np.uint8)

    cnts, hierarchy = cv2.findContours(
            pr_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
        )
    total_granules_det.append(len(cnts))
    total_iou.append(binary_mask_iou(gt_mask, pr_mask))

granules_deviation = np.abs((np.array(total_granules_det) - np.array(processed_data['granules_number']))) / np.array(processed_data['granules_number']) * 100
granules_detected = (np.array(total_granules_det) / np.array(processed_data['granules_number'])) * 100


print(f"Median and Average IoU for all dataset = {np.median(total_iou)} VS {np.average(total_iou)}, disp = {np.std(total_iou)}")
print(f"Median and Average granules devianion for all dataset = {np.median(granules_deviation)} VS {np.average(granules_deviation)}, disp = {np.std(granules_deviation)}")
print(f"Median and Average granules detected for all dataset = {np.median(granules_detected)} VS {np.average(granules_detected)}, disp = {np.std(granules_detected)}")